# 把价格数据和新闻数据的时间序列进行整合

In [1]:
import torch
import pandas as pd

读取被处理过的新闻数据，使用qwen7b进行内容抽取，然后使用jina2进行embedding向量化

In [2]:
df = pd.read_json('/home/userroot/dev/zms/datasets/2024_04_24_index_300_keynotes.jsonl', lines=True)

In [3]:
t = torch.tensor(df['embeddings']).reshape(df.shape[0],1, -1).squeeze()

In [4]:
df.count()

date          29187
code          29187
content       29187
title         29062
key_note      29187
embeddings    29187
dtype: int64

In [5]:
df_embeddings = pd.DataFrame(t)

In [6]:
df_embeddings.insert(0, 'date', df['date'])
df_embeddings.insert(1, 'code', df['code'])



In [7]:
#获得股票代码对应的唯一值token编码
code_uniq = df_embeddings['code'].unique()


#df_code_idx = pd.DataFrame(code_uniq, columns=['code'])


In [8]:

dict_arr = {value: index for index, value in enumerate(code_uniq)}
print(dict_arr)

{'sz300750': 0, 'sh600519': 1, 'sh601318': 2, 'sh600036': 3, 'sz000333': 4, 'sz000858': 5, 'sh600900': 6, 'sh601899': 7, 'sh601166': 8, 'sh600276': 9, 'sh600030': 10, 'sz002594': 11, 'sh601398': 12, 'sz000651': 13, 'sh600887': 14, 'sh601328': 15, 'sz300059': 16, 'sz300760': 17, 'sz000725': 18, 'sh600309': 19, 'sz002475': 20, 'sh600919': 21, 'sz000568': 22, 'sh601288': 23, 'sh601088': 24, 'sz300124': 25, 'sh601816': 26, 'sh600809': 27, 'sz002415': 28, 'sh601012': 29, 'sz002714': 30, 'sh600028': 31}


In [9]:
df_embeddings.head

<bound method NDFrame.head of                      date      code         0         1         2         3  \
0     2024-03-13 08:53:00  sz300750  0.002847 -0.020852  0.011475  0.043627   
1     2023-11-02 09:25:00  sh600519  0.065992 -0.032219  0.054605  0.095922   
2     2023-11-02 09:40:00  sh600519  0.082235 -0.035655  0.053861  0.081966   
3     2023-10-23 11:40:00  sh600519  0.084007 -0.020835  0.011565  0.093759   
4     2023-10-22 08:25:00  sh600519  0.085509 -0.004491  0.017829  0.104517   
...                   ...       ...       ...       ...       ...       ...   
29182 2024-03-12 18:09:00  sz002714  0.073602 -0.013710  0.129248  0.157028   
29183 2024-03-12 18:10:00  sz002714  0.056388 -0.125278 -0.024264  0.172315   
29184 2024-03-12 18:11:00  sz002714  0.193777 -0.032620  0.003392  0.146746   
29185 2024-03-30 03:57:00  sh600028  0.125407 -0.040071 -0.026733  0.048156   
29186 2024-03-30 06:04:00  sh600028  0.089113 -0.019469  0.000637  0.014681   

              4      

In [101]:
import tushare as ts
import numpy as np
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    #sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))
    df_hour = sorted_g['date'].apply(lambda x: x.strftime('%H'))

    sorted_g.insert(1, 'day', df_day) 
    sorted_g.insert(2,'hour', df_hour)

    columns_to_sum = sorted_g.columns[4:]
    grouped_sum = sorted_g.groupby(['day', 'hour'])[columns_to_sum].sum().reset_index()

    # 生成0-24的整数序列
    numbers = list(range(0, 24))
    # 添加前导零，使序列中的每个元素都为两位数
    formatted_numbers = ['{:02}'.format(num) for num in numbers]
    hour_df = pd.DataFrame({'hour' :formatted_numbers})
    hour_df['tmp'] = 1

    day_uniq = pd.DataFrame(grouped_sum['day'].unique(), columns=['day'])
    day_uniq['tmp'] = 1
    day_hour_uniq = pd.merge(day_uniq, hour_df, on='tmp', how='outer')
    day_hour_uniq.pop('tmp')


    # h_group = pd.merge(day_hour_uniq, grouped_sum, on=['day','hour'], how='left')
    # h_group.pop('tmp')

    new_group = pd.merge(day_hour_uniq, grouped_sum, on=['day','hour'], how='left')
    new_group.ffill(inplace=True)
    new_group.bfill(inplace=True)
    break

sh600028


In [105]:
g

,date,code,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
27460,2023-10-17 07:25:00,sh600028,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.000001,-0.066276,-0.045843,0.053201,0.000986,0.074699,-0.077916,0.017354,0.081341,0.010540
27461,2023-10-27 07:20:00,sh600028,0.013855,-0.023100,0.008722,0.041022,-0.006311,0.038489,0.063531,0.051026,...,0.000001,0.039694,-0.066802,0.002989,-0.018051,0.036033,-0.124976,0.025238,0.037902,0.011884
27462,2023-10-27 07:50:00,sh600028,-0.002199,-0.013416,0.019416,0.039308,0.011370,0.025864,0.027132,0.042718,...,0.000001,0.031858,-0.092847,0.004040,-0.019274,0.029769,-0.108654,0.001719,0.039395,0.004930
27463,2023-10-27 10:20:00,sh600028,0.010273,-0.044275,0.011137,0.040191,-0.011078,0.024030,0.046749,0.061205,...,0.000002,-0.010867,-0.111653,0.004817,-0.002668,0.029859,-0.064139,0.002382,0.030754,0.008268
27464,2023-10-27 11:35:00,sh600028,0.007131,-0.038803,0.004082,0.036210,-0.023007,0.066380,0.047236,0.063097,...,0.000002,0.032281,-0.116180,0.043864,-0.043427,0.043635,-0.094472,0.008777,0.071933,0.012413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29142,2024-03-29 20:43:00,sh600028,-0.001330,-0.070001,0.039725,0.070994,0.012530,0.058621,0.045772,-0.016692,...,0.000001,0.051150,0.020012,-0.042361,0.069337,0.018908,-0.053849,0.036583,0.054978,0.023661
29143,2024-03-29 21:53:00,sh600028,0.111759,-0.011510,-0.001408,-0.028649,-0.004101,0.016450,0.044648,0.066162,...,0.000001,0.031524,0.026923,0.025132,-0.004241,0.025384,-0.083795,-0.002135,0.070562,-0.020097
29144,2024-03-29 22:31:00,sh600028,0.101381,-0.061132,-0.037376,0.062716,-0.007989,0.064034,0.120504,-0.040163,...,0.000001,0.015318,-0.055039,-0.073118,0.025159,-0.049565,-0.045515,-0.003627,0.057317,-0.006209
29185,2024-03-30 03:57:00,sh600028,0.125407,-0.040071,-0.026733,0.048156,0.000698,0.006565,0.048651,-0.006650,...,0.000001,0.046349,0.008586,0.028636,-0.005300,0.001943,-0.115601,-0.034833,0.070010,-0.005792


In [109]:
len(grouped_sum['day'].unique())

71

In [111]:
new_group

,day,hour,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,20231017,00,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.000001,-0.066276,-0.045843,0.053201,0.000986,0.074699,-0.077916,0.017354,0.081341,0.010540
1,20231017,01,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.000001,-0.066276,-0.045843,0.053201,0.000986,0.074699,-0.077916,0.017354,0.081341,0.010540
2,20231017,02,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.000001,-0.066276,-0.045843,0.053201,0.000986,0.074699,-0.077916,0.017354,0.081341,0.010540
3,20231017,03,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.000001,-0.066276,-0.045843,0.053201,0.000986,0.074699,-0.077916,0.017354,0.081341,0.010540
4,20231017,04,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.000001,-0.066276,-0.045843,0.053201,0.000986,0.074699,-0.077916,0.017354,0.081341,0.010540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,20240330,19,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,0.000001,0.029354,-0.024381,0.015836,0.007381,-0.001885,-0.089925,-0.020128,0.074156,-0.000846
1700,20240330,20,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,0.000001,0.029354,-0.024381,0.015836,0.007381,-0.001885,-0.089925,-0.020128,0.074156,-0.000846
1701,20240330,21,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,0.000001,0.029354,-0.024381,0.015836,0.007381,-0.001885,-0.089925,-0.020128,0.074156,-0.000846
1702,20240330,22,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,0.000001,0.029354,-0.024381,0.015836,0.007381,-0.001885,-0.089925,-0.020128,0.074156,-0.000846


In [110]:
71*24

1704

In [128]:
import tushare as ts
import numpy as np
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    #sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))
    df_hour = sorted_g['date'].apply(lambda x: x.strftime('%H'))

    sorted_g.insert(1, 'day', df_day) 
    sorted_g.insert(2,'hour', df_hour)

    columns_to_sum = sorted_g.columns[4:]
    grouped_sum = sorted_g.groupby(['day', 'hour'])[columns_to_sum].sum().reset_index()

    # 生成0-24的整数序列
    numbers = list(range(0, 24))
    # 添加前导零，使序列中的每个元素都为两位数
    formatted_numbers = ['{:02}'.format(num) for num in numbers]
    hour_df = pd.DataFrame({'hour' :formatted_numbers})
    hour_df['tmp'] = 1

    day_uniq = pd.DataFrame(grouped_sum['day'].unique(), columns=['day'])
    day_uniq['tmp'] = 1
    day_hour_uniq = pd.merge(day_uniq, hour_df, on='tmp', how='outer')
    day_hour_uniq.pop('tmp')


    # h_group = pd.merge(day_hour_uniq, grouped_sum, on=['day','hour'], how='left')
    # h_group.pop('tmp')

    new_group = pd.merge(day_hour_uniq, grouped_sum, on=['day','hour'], how='left')
    new_group.ffill(inplace=True)
    new_group.bfill(inplace=True)

    min_date = (df_day.min())
    max_date = (df_day.max())

    # 获取最近前后一天的交易日，如果本身是交易日则返回本身
    t_days = pro.trade_cal(exchange='SSE', end_date=min_date)
    last_trade_day = t_days[t_days['is_open']==1].iloc[0]['cal_date']
    t_days = pro.trade_cal(exchange='SSE', start_date=max_date)
    next_trade_day = t_days[t_days['is_open']==1].iloc[-1]['cal_date']
    
    df_kline = pro.daily(ts_code=part2+'.'+part1, start_date=last_trade_day, end_date=next_trade_day)
    #df_kline.to_csv(f'./index_300/{code}_kline.csv',index=False)
    print(f'trade day from {min_date} to {max_date} count: {len(df_kline)}')
    df_kline.sort_values(by='trade_date', inplace=True)
    df_kline['idx'] = idx

    df_kline['tmp'] = 1
    df_kline = pd.merge(df_kline, hour_df, on='tmp', how='outer')
    df_kline.pop('tmp')

    df_kline.rename(columns={'trade_date':'day'}, inplace=True)

    merged_out = pd.merge(new_group, df_kline, on=['day','hour'], how='outer')
    merged_out.ffill(inplace=True)
    merged_out.rename(columns={'day':'date'}, inplace=True)
    merged_out.dropna(inplace=True)

    merged_out.to_csv(f'./index_300_align_hour/{code}.csv',index=False)

    break

sh600028


trade day from 20231017 to 20240330 count: 113


2712

In [129]:
merged_out

,date,hour,0,1,2,3,4,5,6,7,...,open,high,low,close,pre_close,change,pct_chg,vol,amount,idx
0,20231017,00,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,5.86,5.92,5.82,5.91,5.86,0.05,0.8532,982429.17,577921.115,31.0
1,20231017,01,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,5.86,5.92,5.82,5.91,5.86,0.05,0.8532,982429.17,577921.115,31.0
2,20231017,02,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,5.86,5.92,5.82,5.91,5.86,0.05,0.8532,982429.17,577921.115,31.0
3,20231017,03,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,5.86,5.92,5.82,5.91,5.86,0.05,0.8532,982429.17,577921.115,31.0
4,20231017,04,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,5.86,5.92,5.82,5.91,5.86,0.05,0.8532,982429.17,577921.115,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,20240401,19,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,6.39,6.40,6.29,6.36,6.39,-0.03,-0.4695,1265608.18,803452.484,31.0
3260,20240401,20,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,6.39,6.40,6.29,6.36,6.39,-0.03,-0.4695,1265608.18,803452.484,31.0
3261,20240401,21,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,6.39,6.40,6.29,6.36,6.39,-0.03,-0.4695,1265608.18,803452.484,31.0
3262,20240401,22,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,6.39,6.40,6.29,6.36,6.39,-0.03,-0.4695,1265608.18,803452.484,31.0


In [118]:
import tushare as ts
import numpy as np
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    #sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))

    sorted_g['day'] = df_day
    sorted_g.drop('date', axis=1, inplace=True)
    #df_day_news = sorted_g.groupby(['code','day']).sum().reset_index()
    df_day_news = sorted_g.reset_index()
    
    min_date = (df_day.min())
    max_date = (df_day.max())

    t_days = pro.trade_cal(exchange='SSE', end_date=min_date)
    last_trade_day = t_days[t_days['is_open']==1].iloc[0]['cal_date']
    t_days = pro.trade_cal(exchange='SSE', start_date=max_date)
    next_trade_day = t_days[t_days['is_open']==1].iloc[-1]['cal_date']
    
    df_kline = pro.daily(ts_code=part2+'.'+part1, start_date=last_trade_day, end_date=next_trade_day)
    #df_kline.to_csv(f'./index_300/{code}_kline.csv',index=False)
    print(f'trade day from {min_date} to {max_date} count: {len(df_kline)}')
    df_kline.sort_values(by='trade_date', inplace=True)
    df_kline['idx'] = idx

    df_out = pd.merge(df_day_news, df_kline, left_on='day', right_on='trade_date', how='outer')
    
    df_out.pop('code')
    date = df_out.pop('day')
    #df_out.insert(0, 'code', code)
    idx = df_out.pop('idx')
    df_out.insert(1, 'idx', idx)
    df_out.insert(1, 'news_day', date)

    trade_date = df_out.pop('trade_date')
    df_out.insert(0, 'date', trade_date)
    df_out.drop(['ts_code'], axis=1, inplace=True)
    df_out.insert(0, 'all_date', df_out['date'].combine_first(df_out['news_day'])) 
    df_out.sort_values('all_date', ascending=True, inplace=True)
    df_out.pop('date')
    df_out.pop('index')
    df_out.pop('news_day')
    df_out.reset_index(drop=True  , inplace=True)
    df_out.iloc[:, 2:1538] = df_out.iloc[:, 2:1538].ffill()
    
    df_out.iloc[:, -9:] = df_out.iloc[:, -9:].bfill()
    df_out['idx'] = df_out['idx'].ffill()
    df_out.rename(columns={'all_date':'date'}, inplace=True)

    df_out.dropna(inplace=True)
    df_out.to_csv(f'./index_300/{code}.csv',index=False)
    #break

sh600028
trade day from 20231017 to 20240330 count: 113


sh600030
trade day from 20231008 to 20240406 count: 123
sh600036
trade day from 20231028 to 20240406 count: 108
sh600276
trade day from 20231018 to 20240404 count: 115
sh600309
trade day from 20230819 to 20240407 count: 152
sh600519
trade day from 20231022 to 20240406 count: 113
sh600809
trade day from 20231017 to 20240405 count: 116
sh600887
trade day from 20231031 to 20240405 count: 106
sh600900
trade day from 20230706 to 20240403 count: 182
sh600919
trade day from 20230923 to 20240407 count: 127
sh601012
trade day from 20231031 to 20240407 count: 106
sh601088
trade day from 20230914 to 20240407 count: 133
sh601166
trade day from 20231031 to 20240404 count: 106
sh601288
trade day from 20231013 to 20240407 count: 118
sh601318
trade day from 20231028 to 20240405 count: 108
sh601328
trade day from 20231028 to 20240407 count: 108
sh601398
trade day from 20231008 to 20240407 count: 123
sh601816
trade day from 20220919 to 20240407 count: 374
sh601899
trade day from 20231009 to 20240405 cou

In [117]:
df_out

,date,idx,0,1,2,3,4,5,6,7,...,1535,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,20230928,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.07,22.16,21.65,21.66,22.03,-0.37,-1.6795,854425.72,1857947.584
1,20231008,10.0,-0.020890,-0.001655,0.124246,0.044996,0.105124,0.054893,0.093069,0.068659,...,-0.012011,21.42,21.74,21.09,21.68,21.66,0.02,0.0923,707714.49,1518711.509
2,20231009,10.0,-0.020890,-0.001655,0.124246,0.044996,0.105124,0.054893,0.093069,0.068659,...,-0.012011,21.42,21.74,21.09,21.68,21.66,0.02,0.0923,707714.49,1518711.509
3,20231010,10.0,-0.020890,-0.001655,0.124246,0.044996,0.105124,0.054893,0.093069,0.068659,...,-0.012011,21.81,21.86,21.41,21.43,21.68,-0.25,-1.1531,405554.03,875088.891
4,20231011,10.0,-0.020890,-0.001655,0.124246,0.044996,0.105124,0.054893,0.093069,0.068659,...,-0.012011,21.58,21.67,21.45,21.51,21.43,0.08,0.3733,393845.79,848855.684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,20240406,10.0,0.022757,0.017746,0.085960,0.034396,0.110192,0.048601,0.027898,0.028810,...,-0.006102,19.00,19.07,18.78,18.92,19.16,-0.24,-1.2526,724599.54,1371551.741
1060,20240406,10.0,0.061937,-0.024102,0.019762,0.013607,0.092597,-0.013632,-0.000806,0.029540,...,-0.029487,19.00,19.07,18.78,18.92,19.16,-0.24,-1.2526,724599.54,1371551.741
1061,20240406,10.0,0.026316,-0.048804,0.056967,0.009748,0.049678,0.023061,0.005189,-0.008012,...,0.006448,19.00,19.07,18.78,18.92,19.16,-0.24,-1.2526,724599.54,1371551.741
1062,20240406,10.0,0.114627,0.067323,0.027315,-0.004416,0.035878,-0.019099,0.102258,0.065061,...,0.025163,19.00,19.07,18.78,18.92,19.16,-0.24,-1.2526,724599.54,1371551.741


In [107]:
print(df_out.isnull().any().any())

False


In [94]:
df_out.iloc[:, 2:1538] = df_out.iloc[:, 2:1538].fillna(method='ffill')

/tmp/ipykernel_61088/965171644.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_out.iloc[:, 2:1538] = df_out.iloc[:, 2:1538].fillna(method='ffill')


In [97]:
df_out.iloc[:, -9:] = df_out.iloc[:, -9:].fillna(method='bfill')

/tmp/ipykernel_61088/2510493367.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_out.iloc[:, -9:] = df_out.iloc[:, -9:].fillna(method='bfill')


In [98]:
df_out

,all_date,idx,0,1,2,3,4,5,6,7,...,1535,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,20231017,31.0,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.010540,5.86,5.92,5.82,5.91,5.86,0.05,0.8532,982429.17,577921.115
1,20231018,31.0,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.010540,5.93,5.95,5.88,5.90,5.91,-0.01,-0.1692,992642.66,587132.212
2,20231019,31.0,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.010540,5.88,5.88,5.76,5.77,5.90,-0.13,-2.2034,1080748.24,627247.463
3,20231020,31.0,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.010540,5.77,5.78,5.68,5.69,5.77,-0.08,-1.3865,1044869.60,597737.289
4,20231023,31.0,0.017436,-0.037303,-0.001553,0.023647,0.004944,0.062275,0.039161,0.083271,...,0.010540,5.66,5.72,5.51,5.58,5.69,-0.11,-1.9332,1416605.72,793358.266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,20240329,31.0,0.110278,0.016166,-0.035528,0.135906,0.002083,0.067909,0.098000,-0.017859,...,-0.028521,6.21,6.44,6.21,6.39,6.22,0.17,2.7331,1877796.47,1195696.870
919,20240329,31.0,0.111138,-0.012844,-0.026021,0.043651,-0.046420,-0.015820,0.050508,0.027507,...,0.009802,6.21,6.44,6.21,6.39,6.22,0.17,2.7331,1877796.47,1195696.870
920,20240330,NaN,0.125407,-0.040071,-0.026733,0.048156,0.000698,0.006565,0.048651,-0.006650,...,-0.005792,6.39,6.40,6.29,6.36,6.39,-0.03,-0.4695,1265608.18,803452.484
921,20240330,NaN,0.089113,-0.019469,0.000637,0.014681,0.011986,0.010822,0.071112,0.030465,...,-0.000846,6.39,6.40,6.29,6.36,6.39,-0.03,-0.4695,1265608.18,803452.484


In [45]:
df_out['open'].fillna(method = 'bfill')

/tmp/ipykernel_61088/591581578.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_out['open'].fillna(method = 'bfill')


0      5.86
858    5.93
859    5.88
860    5.77
861    5.66
       ... 
836    6.21
835    6.21
854    6.21
857     NaN
856     NaN
Name: open, Length: 922, dtype: float64

In [9]:
import tushare as ts
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    #sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))

    sorted_g['day'] = df_day
    sorted_g.drop('date', axis=1, inplace=True)
    df_day_news = sorted_g.groupby(['code','day']).sum().reset_index()

    min_date = (df_day.min())
    max_date = (df_day.max())
    
    df_kline = pro.daily(ts_code=part2+'.'+part1, start_date=min_date, end_date=max_date)
    print(f'trade day from {min_date} to {max_date} count: {len(df_kline)}')
    df_kline.sort_values(by='trade_date', inplace=True)
    df_kline['idx'] = idx

    df_out = pd.merge(df_kline, df_day_news, left_on='trade_date', right_on='day', how='left')
    df_out.pop('code')
    date = df_out.pop('day')
    #df_out.insert(0, 'code', code)
    idx = df_out.pop('idx')
    df_out.insert(1, 'idx', idx)

    trade_date = df_out.pop('trade_date')
    df_out.insert(0, 'date', trade_date)
    df_out.drop(['ts_code'], axis=1, inplace=True)
    df_out.dropna(inplace=True)
    df_out.to_csv(f'./index_300/{code}.csv',index=False)
    #break

sh600028
trade day count: 112
sh600030
trade day count: 121
sh600036
trade day count: 106
sh600276
trade day count: 114
sh600309
trade day count: 150
sh600519
trade day count: 111
sh600809
trade day count: 115
sh600887
trade day count: 105
sh600900
trade day count: 182
sh600919
trade day count: 125
sh601012
trade day count: 105
sh601088
trade day count: 132
sh601166
trade day count: 105
sh601288
trade day count: 117
sh601318
trade day count: 106
sh601328
trade day count: 106
sh601398
trade day count: 121
sh601816
trade day count: 373
sh601899
trade day count: 121
sz000333
trade day count: 121
sz000568
trade day count: 105
sz000651
trade day count: 105
sz000725
trade day count: 99
sz000858
trade day count: 106
sz002415
trade day count: 230
sz002475
trade day count: 140
sz002594
trade day count: 121
sz002714
trade day count: 104
sz300059
trade day count: 162
sz300124
trade day count: 106
sz300750
trade day count: 16
sz300760
trade day count: 73


In [74]:
df_out

,date,idx,open,high,low,close,pre_close,change,pct_chg,vol,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,20231030,3,31.01,31.13,30.55,30.90,31.53,-0.63,-1.9981,838500.56,...,9.035000e-06,-0.011626,0.133783,-0.302625,0.102012,0.100707,-0.463758,0.087314,0.292798,0.137456
2,20231101,3,30.80,30.88,30.43,30.45,30.76,-0.31,-1.0078,529897.07,...,1.157000e-06,-0.029133,0.017732,-0.041162,0.013725,0.001377,-0.059778,0.016609,0.050532,0.017220
3,20231102,3,30.60,30.81,30.51,30.68,30.45,0.23,0.7553,364667.71,...,1.234600e-06,-0.013911,0.011546,-0.011610,0.001943,0.002128,-0.037798,0.009851,0.036596,0.031129
4,20231103,3,30.68,31.04,30.60,30.99,30.68,0.31,1.0104,439568.79,...,1.275500e-06,-0.004545,0.013368,-0.010489,0.011712,-0.003563,-0.056975,0.032437,0.049904,0.021996
9,20231110,3,30.50,30.56,30.10,30.16,30.66,-0.50,-1.6308,516964.62,...,1.181200e-06,-0.012468,0.017671,-0.008914,0.015629,-0.011193,-0.052474,0.031002,0.049578,0.006951
17,20231122,3,30.30,30.41,30.15,30.16,30.43,-0.27,-0.8873,273705.42,...,1.247700e-06,-0.031123,0.008862,-0.033828,0.023970,0.022787,-0.055320,0.039364,0.052991,0.009940
21,20231128,3,29.41,29.49,29.04,29.16,29.40,-0.24,-0.8163,475165.11,...,1.247300e-06,-0.047621,0.001252,-0.012727,0.033209,-0.015208,-0.062123,0.031082,0.023849,0.009540
58,20240119,3,29.37,29.90,29.14,29.73,29.58,0.15,0.5071,968616.23,...,1.040700e-06,0.014231,0.021930,-0.034369,0.019238,-0.006093,-0.040605,0.003699,0.021148,-0.006606
59,20240122,3,29.76,30.50,29.70,30.25,29.73,0.52,1.7491,1549702.67,...,9.388400e-06,-0.147597,0.235101,-0.189059,0.163584,0.121369,-0.453300,0.249074,0.313732,0.159518
60,20240123,3,30.01,30.35,29.80,29.99,30.25,-0.26,-0.8595,886055.52,...,1.661150e-05,-0.432220,0.492152,-0.472507,0.193099,0.062855,-1.007562,0.181151,0.467811,0.327173


In [32]:
pd.merge(df_kline, df_day_news, left_on='trade_date', right_on='day', how='left').columns.to_list()[:20]

['ts_code',
 'trade_date',
 'open',
 'high',
 'low',
 'close',
 'pre_close',
 'change',
 'pct_chg',
 'vol',
 'amount',
 'code',
 'idx',
 'day',
 0,
 1,
 2,
 3,
 4,
 5]

In [15]:
sorted_g.groupby(['code','idx','day']).sum()

0         1         2         3         4     \
code     idx day                                                          
sh600036 3   20231028  0.052496  0.014271  0.004077 -0.016782  0.019934   
             20231029  0.714024  0.065521  0.019565  0.063348  0.147568   
             20231030  1.013874  0.061956  0.136891  0.040366  0.253617   
             20231101  0.088131  0.008092  0.028570 -0.026598  0.018498   
             20231102  0.171141 -0.020110  0.026719  0.001360 -0.012637   
...                         ...       ...       ...       ...       ...   
             20240402  2.721682  0.770165  0.948058 -0.016964  0.488265   
             20240403  1.421373  0.314657  0.574405 -0.103237  0.444065   
             20240404  0.460762  0.045750  0.255937 -0.077255  0.058724   
             20240405  0.479731  0.039856  0.145834  0.141446  0.015230   
             20240406  0.451252  0.059133  0.297304  0.036926  0.125510   

                           5         6         7         8         9     ...  \
code     idx day                                                         ...   
sh600036 3   20231028  0.042626  0.018953 -0.006233  0.114500  0.008610  ...   
             20231029  0.307323  0.302455  0.076689  0.548733  0.206721  ...   
             20231030  0.478124  0.592757  0.043058  0.953349  0.360063  ...   
             20231101  0.047380  0.016645 -0.021242  0.096557  0.024940  ...   
             20231102  0.071702  0.056196 -0.005677  0.138932  0.031272  ...   
...                         ...       ...       ...       ...       ...  ...   
             20240402  1.523463  2.155370  0.326618  1.497572  0.922950  ...   
             20240403  0.677203  1.390551  0.392631  0.720667  0.254676  ...   
             20240404  0.379485  0.308425  0.066692  0.193518  0.013732  ...   
             20240405  0.304184  0.137109  0.130456  0.231094  0.111042  ...   
             20240406  0.252566  0.455302  0.028752  0.130500  0.043152  ...   

                           1526      1527      1528      1529      1530  \
code     idx day                                                          
sh600036 3   20231028  0.000001 -0.025786  0.029685 -0.026005  0.012603   
             20231029  0.000007 -0.069186  0.049365 -0.166468  0.085142   
             20231030  0.000009 -0.011626  0.133783 -0.302625  0.102012   
             20231101  0.000001 -0.029133  0.017732 -0.041162  0.013725   
             20231102  0.000001 -0.013911  0.011546 -0.011610  0.001943   
...                         ...       ...       ...       ...       ...   
             20240402  0.000037 -1.156892  0.904158 -0.181451  0.524649   
             20240403  0.000024 -0.840107  0.418354 -0.111304  0.486526   
             20240404  0.000007 -0.250288  0.195225 -0.011342  0.101750   
             20240405  0.000006 -0.171141  0.116491 -0.172336  0.051579   
             20240406  0.000006 -0.035822  0.282217 -0.056454  0.180779   

                           1531      1532      1533      1534      1535  
code     idx day                                                         
sh600036 3   20231028  0.009661 -0.055642  0.033669  0.036433  0.016002  
             20231029  0.102239 -0.336227  0.088117  0.194803  0.150933  
             20231030  0.100707 -0.463758  0.087314  0.292798  0.137456  
             20231101  0.001377 -0.059778  0.016609  0.050532  0.017220  
             20231102  0.002128 -0.037798  0.009851  0.036596  0.031129  
...                         ...       ...       ...       ...       ...  
             20240402 -0.547327 -1.878498  0.699713  1.178075  0.913404  
             20240403 -0.641482 -1.437312  0.432396  0.983013  0.741509  
             20240404 -0.154793 -0.368110  0.174599  0.251095  0.174101  
             20240405 -0.008176 -0.378641  0.112632  0.164001  0.123232  
             20240406 -0.175339 -0.509901  0.121158  0.385448  0.128329  

[86 rows x 1536 columns]

In [80]:
import tushare as ts
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))
    min_date = (df_day.min())
    max_date = (df_day.max())
    
    df_kline = pro.daily(ts_code=part2+'.'+part1, start_date=min_date, end_date=max_date)
    # df_day left join df_kline
    df_day = df_day.to_frame()
    df_day.columns = ['trade_date'] 
    df_day = df_day.merge(df_kline, on='trade_date', how='left')

    # cat two dataframe horizontally one is sorted_g and the other is df_day    
    df_all = pd.concat([ sorted_g.reset_index(drop=True), df_day.reset_index(drop=True)], axis=1)
    
    #df_day = df_day.merge(sorted_g, on='idx', how='left')
    #break
    df_all.drop(['trade_date','ts_code'], axis=1, inplace=True)
    df_all.dropna(inplace=True)
    df_all.to_csv(f'./index_300/{code}.csv',index=False)

sh600036
sh600276
sh600519
sh600900
sh601166
sh601318
sh601899
sz000333
sz000858
sz300750


In [78]:
df_all[-6:]

,date,code,idx,0,1,2,3,4,5,6,...,1535,open,high,low,close,pre_close,change,pct_chg,vol,amount
960,2024-04-06 06:39:00,sz300750,0,0.118040,-0.111507,-0.018272,0.080883,0.040546,0.069117,-0.008908,...,0.005173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,2024-04-06 06:39:00,sz300750,0,0.059917,-0.104301,0.056999,0.100061,0.055509,0.009099,0.043320,...,0.058387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,2024-04-06 08:06:00,sz300750,0,0.051554,-0.090702,0.075059,0.056664,0.013563,-0.036048,0.038099,...,0.024600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
963,2024-04-06 09:22:00,sz300750,0,0.028303,-0.061776,0.102021,0.047699,-0.015367,-0.013320,0.010171,...,0.001846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
964,2024-04-06 10:46:00,sz300750,0,-0.059058,0.076216,0.073836,0.230623,0.020653,0.170882,-0.000234,...,-0.082382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965,2024-04-06 14:09:00,sz300750,0,0.037437,0.023296,-0.032922,0.126579,-0.027781,-0.049125,0.005928,...,-0.010793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
#df_all.drop(['trade_date','ts_code'], axis=1, inplace=True)
df_all.columns.to_list()[-40:]

[1505,
 1506,
 1507,
 1508,
 1509,
 1510,
 1511,
 1512,
 1513,
 1514,
 1515,
 1516,
 1517,
 1518,
 1519,
 1520,
 1521,
 1522,
 1523,
 1524,
 1525,
 1526,
 1527,
 1528,
 1529,
 1530,
 1531,
 1532,
 1533,
 1534,
 1535,
 'open',
 'high',
 'low',
 'close',
 'pre_close',
 'change',
 'pct_chg',
 'vol',
 'amount']

In [41]:
print(pd.concat([df_day.reset_index(drop=True), sorted_g.reset_index(drop=True)],axis=1).iloc[7][:])

trade_date     20231030
ts_code       600036.SH
open              31.01
high              31.13
low               30.55
                ...    
1531           0.021719
1532          -0.047426
1533           0.007777
1534           0.055747
1535           0.014372
Name: 7, Length: 1550, dtype: object


In [11]:
sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))

2032    20231028
2033    20231029
2034    20231029
2031    20231029
2035    20231029
          ...   
3967    20240406
3968    20240406
3969    20240406
3970    20240406
3971    20240406
Name: date, Length: 978, dtype: object

In [ ]:
import tushare as ts
pro = ts.pro_api()

df = pro.daily(ts_code='000001.SZ', start_date='20180701', end_date='20180718')